In [5]:
from sklearn.datasets import make_blobs
import pandas as pd
import numpy as np
import math
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
import time
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
from skimage import exposure
import cv2
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
%matplotlib inline

def list_sort_nicely(l):
    def tryint(s):
        try:
            return int(s)
        except:
            return s
    
    def alphanum_key(s):
        return [ tryint(c) for c in re.split('([0-9]+)', s)]
    l.sort(key=alphanum_key)
    return l

1.0.2


In [2]:
train_data_dir = "/mnt/nas203/forGPU/leesangy/final_dataset/train/map_nifti/*.nii.gz"
train_data_list = list_sort_nicely(glob.glob(train_data_dir))

def leave_multiples_of_40(lst):
    return [element for index, element in enumerate(lst) if index % 40 == 0]

train_data_list = leave_multiples_of_40(train_data_list)

print(len(train_data_list))

1065


In [3]:
print(train_data_list[:10])

['/mnt/nas203/forGPU/leesangy/final_dataset/train/map_nifti/001_0000_156_resized.nii.gz', '/mnt/nas203/forGPU/leesangy/final_dataset/train/map_nifti/001_0000_196_resized.nii.gz', '/mnt/nas203/forGPU/leesangy/final_dataset/train/map_nifti/001_0000_236_resized.nii.gz', '/mnt/nas203/forGPU/leesangy/final_dataset/train/map_nifti/001_0000_276_resized.nii.gz', '/mnt/nas203/forGPU/leesangy/final_dataset/train/map_nifti/001_0000_316_resized.nii.gz', '/mnt/nas203/forGPU/leesangy/final_dataset/train/map_nifti/002_0000_156_resized.nii.gz', '/mnt/nas203/forGPU/leesangy/final_dataset/train/map_nifti/002_0000_196_resized.nii.gz', '/mnt/nas203/forGPU/leesangy/final_dataset/train/map_nifti/002_0000_236_resized.nii.gz', '/mnt/nas203/forGPU/leesangy/final_dataset/train/map_nifti/002_0000_276_resized.nii.gz', '/mnt/nas203/forGPU/leesangy/final_dataset/train/map_nifti/002_0000_316_resized.nii.gz']


In [4]:
first_step = np.empty((0,1))

start_time = time.time()

for data_num in range(len(train_data_list)):
    nifti_file = nib.load(train_data_list[data_num])
    nifti_data = np.rot90(nifti_file.get_fdata().squeeze(), k=3)

    nifti_copied = nifti_data.copy()
    background_mask = (nifti_data != 0)

    nifti_data = np.reshape(nifti_data, (256*256,1))
    data = nifti_data[nifti_data!=0]

    data_2d = data.reshape(-1,1)
    
    first_step = np.concatenate((first_step, data_2d), axis=0)
    
    if data_num % 100 == 0:
        print(data_num, (time.time()-start_time)/60, "min")

first_step = sorted(first_step)
print((time.time()-start_time)/60, "min")

In [19]:
print("started")

start_time = time.time()

k=6
kmeans= KMeans(n_clusters=k)
kmeans.fit(first_step)

print("worked", (time.time()-start_time)/60, "min")

labels = kmeans.labels_
centers = kmeans.cluster_centers_

sorted_centers = list(np.sort(centers, axis=None))
centers = []

for i in range(len(sorted_centers)):
    centers.append(sorted_centers[i])

thresholds = []

for i in range(len(centers)):
    threshold = (centers[i]+centers[i+1])/2
    thresholds.append(threshold)
    if i==k-2:
        break
    
centers=thresholds

print(centers)

started
worked 1.3264363328615825 min
[0.145070100337552, 0.26681321049385365, 0.38874155271721045, 0.513746866143867, 0.6587133672077274]


In [16]:
second_step = np.empty((0,1))

start_time = time.time()

for data_num in range(len(train_data_list)):
    nifti_file = nib.load(train_data_list[data_num])
    nifti_data = np.rot90(nifti_file.get_fdata().squeeze(), k=3)

    nifti_copied = nifti_data.copy()
    background_mask = (nifti_data != 0)    
    nifti_copied[nifti_copied>centers[0]]=0
    nifti_copied_data = np.reshape(nifti_copied, (256*256,1))
    data = nifti_copied_data[nifti_copied_data!=0]
    data_2d = data.reshape(-1,1)
    
    second_step = np.concatenate((second_step, data_2d), axis=0)
    
    if data_num % 100 == 0:
        print(data_num, (time.time()-start_time)/60, "min")

0 7.385810216267904e-05 min
100 0.0040116389592488606 min
200 0.00842660665512085 min
300 0.013129484653472901 min
400 0.018595691521962485 min
500 0.04030586083730062 min
600 0.04814697504043579 min
700 0.07908655405044555 min
800 0.10756194591522217 min
900 0.13429862260818481 min
1000 0.1521301507949829 min


In [17]:
second_step = sorted(second_step)

In [18]:
k=4
kmeans= KMeans(n_clusters=k)
kmeans.fit(second_step)
labels = kmeans.labels_
centers = kmeans.cluster_centers_
sorted_centers = list(np.sort(centers, axis=None))
centers = []

for i in range(len(sorted_centers)):
    centers.append(sorted_centers[i])

new_threshold = (centers[1]+centers[2])/2
thresholds.append(new_threshold)
centers = sorted(thresholds)


print(centers)

[0.09204364444352381, 0.17063122017218094, 0.31581678663409585, 0.45899430876793795, 0.6147709754131994]
